In [1]:
import pandas, warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

plt.style.use("seaborn-whitegrid")

In [2]:
reader = lambda url, **multiargs : pandas.read_table(url, sep=",", **multiargs)

tanggal = ["Date"]

url = "/content/online_retail.csv"

origin = reader(url, parse_dates=tanggal)

origin.iloc[:5, :4]

,Invoice,Stock,Description,Quantity
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12
1,489434,79323P,PINK CHERRY LIGHTS,12
2,489434,79323W,WHITE CHERRY LIGHTS,12
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24


In [3]:
origin.iloc[:5, 4:]

,Date,Price,Customer,Country
0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
null = origin.isnull().sum()

null.sum()

0

### ```Single```

In [5]:
oten = origin[origin["Date"].dt.year == 2011]

oten["Date"].dt.year.unique().tolist()

[2011]

In [6]:
import datetime

seter = lambda i : datetime.datetime(i.year, i.month, 1)

oten["OrderMonth"] = oten["Date"].apply(seter)

grouper = oten.groupby("Customer")["OrderMonth"]

In [7]:
oten["CustomerMonth"] = grouper.transform("min")

oten.iloc[:5, 6:]

,Customer,Country,OrderMonth,CustomerMonth
433855,13313.0,United Kingdom,2011-01-01,2011-01-01
433856,13313.0,United Kingdom,2011-01-01,2011-01-01
433857,13313.0,United Kingdom,2011-01-01,2011-01-01
433858,13313.0,United Kingdom,2011-01-01,2011-01-01
433859,13313.0,United Kingdom,2011-01-01,2011-01-01


In [8]:
oten[oten["Customer"] == 13313.0]["CustomerMonth"].nunique()

1

In [9]:
def getime(cols): return cols.dt.year, cols.dt.month

order_year, order_month = getime(oten["OrderMonth"])

custear, custom = getime(oten["CustomerMonth"])

orit = order_year - custear

orti = order_month - custom

oten["Cluster"] = orit * 12 + orti + 1

oten.iloc[:5, 6:]

,Customer,Country,OrderMonth,CustomerMonth,Cluster
433855,13313.0,United Kingdom,2011-01-01,2011-01-01,1
433856,13313.0,United Kingdom,2011-01-01,2011-01-01,1
433857,13313.0,United Kingdom,2011-01-01,2011-01-01,1
433858,13313.0,United Kingdom,2011-01-01,2011-01-01,1
433859,13313.0,United Kingdom,2011-01-01,2011-01-01,1


In [10]:
oten["Cluster"].unique().tolist()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [ ]:
grouper = oten.groupby(["CustomerMonth", "Cluster"])

grouper["Customer"].apply(pandas.Series.nunique).iloc[:5]

In [12]:
customer = grouper["Customer"].apply(pandas.Series.nunique)

customer = customer.reset_index()

customer.iloc[:5]

,CustomerMonth,Cluster,Customer
0,2011-01-01,1,741
1,2011-01-01,2,262
2,2011-01-01,3,319
3,2011-01-01,4,284
4,2011-01-01,5,339


In [13]:
custom = customer.pivot(index="CustomerMonth", columns="Cluster", values="Customer")

custom.iloc[:5, :5]

Cluster,1,2,3,4,5
CustomerMonth,,,,,
2011-01-01,741.0,262.0,319.0,284.0,339.0
2011-02-01,496.0,116.0,123.0,169.0,155.0
2011-03-01,539.0,88.0,144.0,120.0,123.0
2011-04-01,361.0,85.0,79.0,85.0,76.0
2011-05-01,319.0,58.0,56.0,58.0,70.0


In [14]:
custi = custom.iloc[:, 0]

retention = custom.divide(custi, axis="rows")

retention = retention.round(3) * 100

retention.iloc[:5, :5]

Cluster,1,2,3,4,5
CustomerMonth,,,,,
2011-01-01,100.0,35.4,43.0,38.3,45.7
2011-02-01,100.0,23.4,24.8,34.1,31.2
2011-03-01,100.0,16.3,26.7,22.3,22.8
2011-04-01,100.0,23.5,21.9,23.5,21.1
2011-05-01,100.0,18.2,17.6,18.2,21.9


In [15]:
grouper = oten.groupby(["CustomerMonth", "Cluster"])

quantitas = grouper["Quantity"].mean()

quantitas = quantitas.reset_index()

quantitas = quantitas.pivot(index="CustomerMonth", columns="Cluster", values="Quantity").round(1)

quantitas.iloc[:5, :5]

Cluster,1,2,3,4,5
CustomerMonth,,,,,
2011-01-01,16.4,15.5,15.7,14.6,15.3
2011-02-01,11.9,16.7,17.6,13.3,14.9
2011-03-01,9.8,13.5,12.4,9.5,13.9
2011-04-01,9.6,9.7,9.3,11.4,11.6
2011-05-01,10.9,10.2,12.6,12.6,10.8


### ```Quarter```